<a href="https://colab.research.google.com/github/Odeke12/machine_learning/blob/main/Artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X[0])

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]


### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X[0])

[619 'France' 0 42 2 0.0 1 1 1 101348.88]


One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [ ]:
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


In [ ]:
print(X[3])

[1.0 0.0 0.0 699 0 39 1 0.0 2 0 0 93826.63]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train[0])

[-0.99850112  1.71490137 -0.57273139 -0.23082038  0.91509065 -0.94449979
 -0.70174202  0.58817274  0.80225696 -1.55337352  0.97725852  0.42739449]


## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = "adam", loss="binary_crossentropy" , metrics=['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
250/250 [==============================] - 1s 1ms/step - loss: 0.7718 - accuracy: 0.5088
Epoch 2/20
250/250 [==============================] - 0s 1ms/step - loss: 0.5157 - accuracy: 0.7965
Epoch 3/20
250/250 [==============================] - 0s 1ms/step - loss: 0.4326 - accuracy: 0.8176
Epoch 4/20
250/250 [==============================] - 0s 1ms/step - loss: 0.4216 - accuracy: 0.8104
Epoch 5/20
250/250 [==============================] - 0s 1ms/step - loss: 0.4018 - accuracy: 0.8233
Epoch 6/20
250/250 [==============================] - 0s 1ms/step - loss: 0.3844 - accuracy: 0.8370
Epoch 7/20
250/250 [==============================] - 0s 1ms/step - loss: 0.3756 - accuracy: 0.8427
Epoch 8/20
250/250 [==============================] - 0s 1ms/step - loss: 0.3687 - accuracy: 0.8479
Epoch 9/20
250/250 [==============================] - 0s 1ms/step - loss: 0.3619 - accuracy: 0.8498
Epoch 10/20
250/250 [==============================] - 0s 1ms/step - loss: 0.3491 - accuracy: 0.8585

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [ ]:
#eval = sc.transform(['France',600,'Male',60000,2,1,2,50000])
print(ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 6000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

### Making the Confusion Matrix

In [ ]:
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1566   19]
 [ 306  109]]


0.8375